In [15]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import LeaveOneOut
from transformers import DistilBertModel, DistilBertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold


data = pd.read_csv('mohler_dataset_edited.csv')

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenize and prepare data
def tokenize_data(data, max_seq_length=128):
    input_ids = []
    attention_masks = []
    scores_list = []

    for i in range(len(data)):
        answer1_tokens = tokenizer.encode(data['desired_answer'].iloc[i], add_special_tokens=True)
        answer2_tokens = tokenizer.encode(data['student_answer'].iloc[i], add_special_tokens=True)

        # Truncate or pad the tokens to the specified max sequence length
        answer1_tokens = answer1_tokens[:max_seq_length-2]  # -2 for [CLS] and [SEP] tokens
        answer2_tokens = answer2_tokens[:max_seq_length-2]  # -2 for [CLS] and [SEP] tokens

        # Add [CLS] and [SEP] tokens
        answer1_tokens = [101] + answer1_tokens + [102]
        answer2_tokens = [101] + answer2_tokens + [102]

        # Padding
        padding_length = max_seq_length - len(answer1_tokens)
        padded_answer1 = answer1_tokens + [0] * padding_length
        padding_length = max_seq_length - len(answer2_tokens)
        padded_answer2 = answer2_tokens + [0] * padding_length

        input_ids.append(padded_answer1)
        attention_masks.append([1] * len(padded_answer1))

        input_ids.append(padded_answer2)
        attention_masks.append([1] * len(padded_answer2))

        # Append the score twice for both answer1 and answer2
        scores_list.append(data['score_avg'].iloc[i])
        scores_list.append(data['score_avg'].iloc[i])

    return torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(scores_list, dtype=torch.float32)

In [16]:
class DistilBertRegression(nn.Module):
    def __init__(self):
        super(DistilBertRegression, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.linear = nn.Linear(self.distilbert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0, :]  
        return self.linear(pooled_output)

In [18]:
learning_rate = 2e-5
batch_size = 16
epochs = 12
weight_decay = 0.01  

kf = KFold(n_splits=10, shuffle=True)

models = {
    'DistilBERT': DistilBertRegression,
    # Add RoBERTa and BERT models here
}

evaluation_results = []

shuffled_data = data.sample(frac=1).reset_index(drop=True)

for model_name, ModelClass in models.items():  
    print(f"Evaluating using {model_name} model...")

    total_rmse = 0
    total_mae = 0
    total_nrmse = 0

    fold_rmse = []
    fold_mae = []
    fold_nrmse = []

    for fold_idx, (train_index, val_index) in enumerate(kf.split(shuffled_data), 1):
        train_data = shuffled_data.iloc[train_index]
        val_data = shuffled_data.iloc[val_index]

        train_input_ids, train_attention_masks, train_scores = tokenize_data(train_data)
        val_input_ids, val_attention_masks, val_scores = tokenize_data(val_data)

        train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_scores)
        train_sampler = RandomSampler(train_dataset)
        train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=16)

        val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_scores)
        val_sampler = SequentialSampler(val_dataset)
        val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=16)

        model = ModelClass().to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)  

        for epoch in tqdm(range(epochs), desc=f"Fold {fold_idx}/{kf.get_n_splits()}"):  
            model.train()
            total_loss = 0

            for batch in train_dataloader:
                batch = tuple(t.to(device) for t in batch)
                inputs, masks, labels = batch

                optimizer.zero_grad()
                outputs = model(inputs, masks)
                loss = criterion(outputs.squeeze(-1), labels)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            avg_train_loss = total_loss / len(train_dataloader)

            model.eval()
            all_predictions = []
            all_labels = []
            with torch.no_grad():
                for val_batch in val_dataloader:
                    val_batch = tuple(t.to(device) for t in val_batch)
                    val_inputs, val_masks, val_labels = val_batch
                    val_outputs = model(val_inputs, val_masks)
                    val_predictions = val_outputs.squeeze(-1).detach().cpu().numpy()
                    all_predictions.extend(val_predictions)
                    all_labels.extend(val_labels.detach().cpu().numpy())

                val_rmse = np.sqrt(mean_squared_error(all_labels, all_predictions))
                val_mae = mean_absolute_error(all_labels, all_predictions)
                val_nrmse = val_rmse / (max(all_labels) - min(all_labels))  

                print(f"Fold {fold_idx}/{kf.get_n_splits()}: Epoch {epoch + 1}/{epochs}, RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}, NRMSE: {val_nrmse:.4f}")

            fold_rmse.append(val_rmse)
            fold_mae.append(val_mae)
            fold_nrmse.append(val_nrmse)

    avg_rmse = np.mean(fold_rmse)
    avg_mae = np.mean(fold_mae)
    avg_nrmse = np.mean(fold_nrmse)

    model_eval_results = {
        f'{model_name}_RMSE': avg_rmse,
        f'{model_name}_MAE': avg_mae,
        f'{model_name}_NRMSE': avg_nrmse
    }
    evaluation_results.append(model_eval_results)

df = pd.DataFrame(evaluation_results)
print(df)


Evaluating using DistilBERT model...


Fold 1/10:   8%|▊         | 1/12 [00:53<09:43, 53.02s/it]

Fold 1/10: Epoch 1/12, RMSE: 1.2094, MAE: 1.0254, NRMSE: 0.2419


Fold 1/10:  17%|█▋        | 2/12 [01:44<08:40, 52.05s/it]

Fold 1/10: Epoch 2/12, RMSE: 1.1849, MAE: 0.9639, NRMSE: 0.2370


...

Fold 10/10: Epoch 10/12, RMSE: 1.0570, MAE: 0.7730, NRMSE: 0.2114


Fold 10/10:  92%|█████████▏| 11/12 [09:32<00:52, 52.07s/it]

Fold 10/10: Epoch 11/12, RMSE: 0.9931, MAE: 0.6722, NRMSE: 0.1986


Fold 10/10: 100%|██████████| 12/12 [10:24<00:00, 52.03s/it]

Fold 10/10: Epoch 12/12, RMSE: 1.0042, MAE: 0.6846, NRMSE: 0.2008
   DistilBERT_RMSE  DistilBERT_MAE  DistilBERT_NRMSE
0         1.060811        0.808247          0.225245
